### 실습 목차
* 1. Transformer 모델 구현
  * 1-1. Multi-head attention 구현
  * 1-2. Feed-forward network 구현
  * 1-3. Positional Encoding 구현
  * 1-4. Encoder Layer 구현
  * 1-5. Decoder Layer 구현
  * 1-6. Transformer 모델 구현
* 2. Transformer 모델 학습
  * 2-1. 학습 데이터 준비
  * 2-2. Loss function과 Optimizer 설정
  * 2-3. Transformer 모델 학습

In [1]:
# 라이브러리 import
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import math
import copy

# GPU 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## 1. Transformer 모델 구현

```
💡 목차 개요 : pytorch를 통해 Transformer의 구성요소들을 각각 구현하고, 결합하여 Transformer의 동작 원리를 이해합니다
```

- 1-1. Multi-head attention 구현
- 1-2. Feed-forward network 구현
- 1-3. Positional Encoding 구현
- 1-4. Encoder Layer 구현
- 1-5. Decoder Layer 구현
- 1-6. Transformer 모델 구현


##### Multi-head attention 구현

MultiHeadAttention은 Transformer 모델에서 사용되는 어텐션 메커니즘입니다. 여러 개의 어텐션 헤드를 사용하여 입력 시퀀스의 다양한 관점에서 정보를 수집합니다


<img src="https://github.com/js-lee-AI/js-lee-AI/assets/60927808/2af71df4-89ae-4d3c-9c06-b9c9bbeeeafb" height="400">

`MultiHeadAttention` 클래스는 다음과 같이 구현됩니다:

- `__init__` : 초기화 함수에서는 `model_dim`, `num_heads`를 입력으로 받습니다. `model_dim`은 모델의 차원을 나타내며, `num_heads`는 사용할 어텐션 헤드의 개수를 나타냅니다. 이때, `model_dim`은 `num_heads`로 나누어 떨어져야 합니다. 각각의 선형 변환을 위한 Linear 레이어를 정의합니다

- `scaled_dot_product_attention` : 스케일드 닷 프로덕트 어텐션을 수행하는 함수입니다. `query`, `key`, `value`를 입력으로 받아 어텐션 스코어를 계산하고, 어텐션 확률을 구한 후, 출력을 반환합니다. 필요한 경우, `mask`를 사용하여 특정 위치의 어텐션 값을 무시할 수 있습니다

- `split_heads` : 입력 텐서를 어텐션 헤드 수에 맞게 분할하는 함수입니다. 배치 크기, 시퀀스 길이, 모델 차원을 구한 후, 텐서의 형태를 변경하여 어텐션 헤드를 분할합니다

- `combine_heads` : 분할된 어텐션 헤드를 다시 결합하는 함수입니다. 배치 크기, 시퀀스 길이, 키 차원을 구한 후, 텐서의 형태를 변경하여 어텐션 헤드를 결합합니다.

- `forward` : MultiHeadAttention의 순전파 함수입니다. `query`, `key`, `value`를 입력으로 받아 선형 변환을 수행한 후, 어텐션 헤드를 분할하고 스케일드 닷 프로덕트 어텐션을 수행합니다. 어텐션 출력을 어텐션 헤드로 결합한 후, 최종 선형 변환을 수행하여 결과를 반환합니다. 필요한 경우, `mask`를 사용하여 특정 위치의 어텐션 값을 무시할 수 있습니다




In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, model_dim, num_heads):
        super(MultiHeadAttention, self).__init__()
        # 모델 차원이 헤드 수로 나누어 떨어지지 않으면 에러 발생
        assert model_dim % num_heads == 0, "model_dim must be divisible by num_heads"

        # 입력받은 값을 인스턴스 변수로 저장
        self.model_dim = model_dim
        self.num_heads = num_heads
        self.key_dim = model_dim // num_heads ## d_k = d_model // num_heads

        # 선형 변환을 위한 Linear 레이어를 정의
        self.query_linear = nn.Linear(model_dim, model_dim)
        self.key_linear = nn.Linear(model_dim, model_dim)
        self.value_linear = nn.Linear(model_dim, model_dim)
        self.output_linear = nn.Linear(model_dim, model_dim)

    # Scaled Dot-Product Attention을 계산하는 함수
    def scaled_dot_product_attention(self, query, key, value, mask=None):
        # Query와 Key의 행렬 곱을 계산하고, Key 차원의 제곱근으로 나눠줌
        attention_scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(self.key_dim)

        # 마스크가 주어진 경우, 마스크가 0인 위치에 매우 작은 값을 채워줌
        if mask is not None:
            attention_scores = attention_scores.masked_fill(mask == 0, -1e9)

        # Softmax를 적용해 Attention 확률을 계산
        attention_probs = torch.softmax(attention_scores, dim=-1)

        # Attention 확률과 Value의 행렬 곱을 계산
        output = torch.matmul(attention_probs, value)

        return output

    # 입력 Tensor를 헤드 수만큼 분할
    def split_heads(self, tensor):
        batch_size, seq_length, model_dim = tensor.size()
        return tensor.view(batch_size, seq_length, self.num_heads, self.key_dim).transpose(1, 2)

    # 분할된 헤드를 다시 합침
    def combine_heads(self, tensor):
        batch_size, _, seq_length, key_dim = tensor.size()
        return tensor.transpose(1, 2).contiguous().view(batch_size, seq_length, self.model_dim)

    # 순전파 정의
    def forward(self, query, key, value, mask=None):

        # Query, Key, Value에 각각 선형 변환을 적용하고 헤드를 분할
        query = self.split_heads(self.query_linear(query))
        key = self.split_heads(self.key_linear(key))
        value = self.split_heads(self.value_linear(value))

        # Scaled Dot-Product Attention을 계산
        attention_output = self.scaled_dot_product_attention(query, key, value, mask)

        # 분할된 헤드를 다시 합치고, 선형 변환을 적용
        output = self.output_linear(self.combine_heads(attention_output))
        
        return output


##### Feed-forward network 구현

FeedForwardNN는 Transformer 모델의 각 레이어에서 사용되는 Feed-Forward 신경망입니다. 이 구조는 입력 시퀀스의 각 위치에 독립적으로 적용되며, 동일한 가중치를 공유합니다

![image](https://github.com/js-lee-AI/optimal-path-search_imitation-learning/assets/60927808/6b460433-95e7-4c0f-8e5f-e77d68937650)


FeedForwardNN 클래스는 다음과 같이 구현됩니다:

- `__init__` 함수에서 두 개의 선형 레이어(`nn.Linear`)를 초기화합니다. 첫 번째 선형 레이어는 입력 벡터의 차원을 `model_dim`에서 `feedforward_dim`으로 변환하고, 두 번째 선형 레이어는 `feedforward_dim`에서 `model_dim`으로 다시 변환합니다. 또한, 활성화 함수로 ReLU(`nn.ReLU`)를 사용합니다

- `forward` 함수에서 입력 벡터 `x`를 첫 번째 선형 레이어와 ReLU 활성화 함수를 통과시킨 후, 두 번째 선형 레이어를 통과시켜 결과를 반환합니다. 이를 통해 입력 벡터의 차원을 일시적으로 확장하고 다시 축소하는 과정을 수행합니다

In [ ]:
class FeedForwardNN(nn.Module):
    def __init__(self, model_dim, feedforward_dim):
        super(FeedForwardNN, self).__init__()
        # 선형 변환을 위한 Linear 레이어와 활성화 함수 정의
        self.fc1 = nn.Linear(model_dim, feedforward_dim)
        self.fc2 = nn.Linear(feedforward_dim, model_dim)
        self.relu = nn.ReLU()

    # 순전파 정의
    def forward(self, x):
        return self.fc2(self.relu(self.fc1(x)))

##### Positional Encoding 구현

Positional Encoding은 Transformer 모델에서 입력 시퀀스의 위치 정보를 제공하는 메커니즘입니다. 이를 통해 모델이 각 입력 단어의 위치에 대한 정보를 인식하고 처리할 수 있습니다

<img src="https://github.com/js-lee-AI/js-lee-AI/assets/60927808/4cea42db-d3dc-4214-a9c3-c4580c35dfa1" height="550">

`PositionalEncoding` 클래스는 다음과 같이 구현됩니다:

- `__init__` 함수에서는 `model_dim`과 `max_seq_length`를 인자로 받습니다. `model_dim`은 모델의 차원을, `max_seq_length`는 최대 시퀀스 길이를 나타냅니다. 이 함수에서는 `positional_encoding` 텐서를 초기화하고, sin과 cos 함수를 사용하여 위치 정보를 인코딩합니다

- `forward` 함수에서는 입력 텐서 `x`와 `positional_encoding`을 더하여 위치 정보가 포함된 텐서를 반환합니다. 이렇게 함으로써, 모델은 입력 시퀀스의 각 단어에 대한 위치 정보를 인식할 수 있습니다

- 짝수 인덱스에 대한 수식:
$PE_{(pos, 2i)} = sin(pos / 10000^{2i/d_{model}})$

- 홀수 인덱스에 대한 수식:
$PE_{(pos, 2i+1)} = cos(pos / 10000^{2i/d_{model}})$

<img src="https://github.com/js-lee-AI/js-lee-AI/assets/60927808/5a266edb-5006-407c-8ce2-ea6d34dc51ae" height="400">


In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self, model_dim, max_seq_length):
        super(PositionalEncoding, self).__init__()

        # Positional Encoding을 계산하기 위한 빈 텐서 생성
        # 이 텐서의 크기는 (max_seq_length, model_dim)임
        positional_encoding = torch.zeros(max_seq_length, model_dim)

        # 각 위치(position)에 대한 정보를 담은 텐서를 생성
        # 이 텐서의 크기는 (max_seq_length, 1)임
        position = torch.arange(0, max_seq_length, dtype=torch.float).unsqueeze(1)

        # 각 차원(dimension)에 대한 분모(div_term)를 계산
        # 이 텐서의 크기는 (model_dim // 2,)임
        div_term = torch.exp(torch.arange(0, model_dim, 2).float() * -(math.log(10000.0) / model_dim))

        # 짝수 인덱스에는 sin 함수를 적용
        # 짝수 인덱스에 대한 수식: PE(pos, 2i) = sin(pos / 10000^(2i/d_model))
        positional_encoding[:, 0::2] = torch.sin(position * div_term)

        # 홀수 인덱스에는 cos 함수를 적용
        # 홀수 인덱스에 대한 수식: PE(pos, 2i+1) = cos(pos / 10000^(2i/d_model))
        positional_encoding[:, 1::2] = torch.cos(position * div_term)


        # 계산된 Positional Encoding을 모듈의 버퍼에 등록
        # 이를 통해 모듈의 state_dict에 포함되어 저장 및 로딩이 가능해짐
        self.register_buffer('positional_encoding', positional_encoding.unsqueeze(0))

    # 순전파 함수를 정의
    def forward(self, x):
        # 입력 x와 Positional Encoding을 더하여 반환
        # 이 때, 입력 x의 시퀀스 길이에 맞추어 Positional Encoding을 슬라이싱함
        return x + self.positional_encoding[:, :x.size(1)]

##### Encoder Layer 구현

`EncoderLayer` 클래스는 Transformer 모델의 인코더 레이어를 구현합니다. 각 인코더 레이어는 self-attention 메커니즘과 feed-forward 신경망을 포함하며, 이를 통해 입력 시퀀스의 정보를 처리합니다

<img src="https://github.com/js-lee-AI/js-lee-AI/assets/60927808/0d004081-ae47-4de1-8aa9-d436c09d91f3" height="400">

- `__init__` 함수에서는 `model_dim`, `num_heads`, `feedforward_dim`, `dropout`을 인자로 받습니다. `model_dim`은 모델의 차원을, `num_heads`는 multi-head attention의 헤드 수를, `feedforward_dim`은 feed-forward 신경망의 차원을, `dropout`은 드롭아웃 비율을 나타냅니다. 이 함수에서는 `MultiHeadAttention`, `FeedForwardNN`, 그리고 두 개의 Layer Normalization 계층을 초기화합니다

- `forward` 함수에서는 입력 텐서 `x`와 마스크 텐서 `mask`를 인자로 받습니다. 먼저, self-attention 메커니즘을 적용한 후, Layer Normalization과 드롭아웃을 적용합니다. 그 다음, feed-forward 신경망을 적용하고 다시 Layer Normalization과 드롭아웃을 적용하여 최종 결과를 반환합니다



In [ ]:
class EncoderLayer(nn.Module):
    def __init__(self, model_dim, num_heads, feedforward_dim, dropout):
        super(EncoderLayer, self).__init__()
        # MultiHeadAttention과 FeedForwardNN 정의
        self.self_attention = MultiHeadAttention(model_dim, num_heads)
        self.feed_forward = FeedForwardNN(model_dim, feedforward_dim)
        # Layer Normalization과 Dropout 정의
        self.norm1 = nn.LayerNorm(model_dim)
        self.norm2 = nn.LayerNorm(model_dim)
        self.dropout = nn.Dropout(dropout)

    # 순전파 정의
    def forward(self, x, mask):
        # Self-Attention을 계산하고, 결과를 원래의 입력과 더한 후 Layer Normalization 적용
        attention_output = self.self_attention(x, x, x, mask)
        x = self.norm1(x + self.dropout(attention_output))
        # Feedforward를 계산하고, 결과를 원래의 입력과 더한 후 Layer Normalization 적용
        feedforward_output = self.feed_forward(x)
        x = self.norm2(x + self.dropout(feedforward_output))

        return x

##### Decoder Layer 구현

`DecoderLayer` 클래스는 Transformer 모델의 디코더 레이어를 구현합니다. 각 디코더 레이어는 두 개의 self-attention 메커니즘(하나는 디코더 입력에, 다른 하나는 인코더 출력에 적용)과 feed-forward 신경망을 포함하며, 이를 통해 인코더에서 전달된 정보와 함께 출력 시퀀스를 생성합니다


<img src="https://github.com/js-lee-AI/js-lee-AI/assets/60927808/d470e33e-54d7-4a89-a369-4851ee4ae428" height="600">


- `__init__` 함수에서는 `model_dim`, `num_heads`, `feedforward_dim`, `dropout`을 인자로 받습니다. `model_dim`은 모델의 차원을, `num_heads`는 multi-head attention의 헤드 수를, `feedforward_dim`은 feed-forward 신경망의 차원을, `dropout`은 드롭아웃 비율을 나타냅니다. 이 함수에서는 두 개의 `MultiHeadAttention` (하나는 self-attention, 다른 하나는 cross-attention), `FeedForwardNN`, 그리고 세 개의 Layer Normalization 계층을 초기화합니다

- `forward` 함수에서는 입력 텐서 `x`, 인코더 출력 텐서 `encoder_output`, 소스 마스크 텐서 `src_mask`, 타겟 마스크 텐서 `tgt_mask`를 인자로 받습니다. 먼저, self-attention 메커니즘을 적용한 후, Layer Normalization과 드롭아웃을 적용합니다. 그 다음, cross-attention 메커니즘을 적용하고 다시 Layer Normalization과 드롭아웃을 적용합니다. 마지막으로, feed-forward 신경망을 적용하고 Layer Normalization과 드롭아웃을 적용하여 최종 결과를 반환합니다

In [ ]:
class DecoderLayer(nn.Module):
    def __init__(self, model_dim, num_heads, feedforward_dim, dropout):
        super(DecoderLayer, self).__init__()
        # MultiHeadAttention과 FeedForwardNN 정의
        self.self_attention = MultiHeadAttention(model_dim, num_heads)
        self.cross_attention = MultiHeadAttention(model_dim, num_heads)
        self.feed_forward = FeedForwardNN(model_dim, feedforward_dim)
        # Layer Normalization과 Dropout을 정의
        self.norm1 = nn.LayerNorm(model_dim)
        self.norm2 = nn.LayerNorm(model_dim)
        self.norm3 = nn.LayerNorm(model_dim)
        self.dropout = nn.Dropout(dropout)

    # 순전파 정의
    def forward(self, x, encoder_output, src_mask, tgt_mask):
        # Self-Attention을 계산하고, 결과를 원래의 입력과 더한 후 Layer Normalization을 적용
        attention_output = self.self_attention(x, x, x, tgt_mask)
        x = self.norm1(x + self.dropout(attention_output))
        # Cross-Attention을 계산하고, 결과를 원래의 입력과 더한 후 Layer Normalization을 적용
        attention_output = self.cross_attention(x, encoder_output, encoder_output, src_mask)
        x = self.norm2(x + self.dropout(attention_output))
        # Feedforward를 계산하고, 결과를 원래의 입력과 더한 후 Layer Normalization을 적용
        feedforward_output = self.feed_forward(x)
        x = self.norm3(x + self.dropout(feedforward_output))

        return x


##### Transformer 모델 구현

`Transformer` 클래스는 전체 Transformer 모델을 구현합니다. 이 클래스는 인코더와 디코더를 포함하며, 각 레이어에서 처리된 정보를 통해 최종 출력 시퀀스를 생성합니다

<img src="https://github.com/js-lee-AI/js-lee-AI/assets/60927808/60d5d857-899e-480a-8ffa-fedb6aeeefd5" height="600">

- `__init__` 함수에서는 `src_vocab_size`, `tgt_vocab_size`, `model_dim`, `num_heads`, `num_layers`, `feedforward_dim`, `max_seq_length`, `dropout`을 인자로 받습니다. 각 인자는 소스와 타겟 어휘 사전의 크기, 모델의 차원, multi-head attention의 헤드 수, 인코더와 디코더 레이어의 수, feed-forward 신경망의 차원, 최대 시퀀스 길이, 드롭아웃 비율을 나타냅니다. 이 함수에서는 인코더와 디코더의 임베딩 계층, 위치 인코딩 계층, 인코더와 디코더 레이어들, 그리고 최종 선형 계층을 초기화합니다

- `generate_mask` 함수에서는 입력 텐서 `src`와 `tgt`를 사용하여 마스크 텐서를 생성합니다. 소스 마스크는 입력 시퀀스에서 패딩 토큰을 가리고, 타겟 마스크는 패딩 토큰과 미래의 토큰을 가립니다. 이렇게 생성된 마스크 텐서들은 인코더와 디코더 레이어에서 사용됩니다

- `forward` 함수에서는 입력 텐서 `src`와 `tgt`를 인자로 받습니다. 먼저, 마스크 텐서를 생성한 후, 인코더와 디코더의 임베딩 계층과 위치 인코딩 계층을 적용합니다. 그 다음, 인코더 레이어들을 차례대로 적용하여 인코더 출력을 얻습니다. 이 인코더 출력과 함께 디코더 레이어들을 차례대로 적용하여 디코더 출력을 얻습니다. 마지막으로, 최종 선형 계층을 적용하여 출력 시퀀스를 반환합니다

In [ ]:
class Transformer(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, model_dim, num_heads, num_layers, feedforward_dim, max_seq_length, dropout):
        super(Transformer, self).__init__()
        # Embedding과 Positional Encoding 정의
        self.encoder_embedding = nn.Embedding(src_vocab_size, model_dim)
        self.decoder_embedding = nn.Embedding(tgt_vocab_size, model_dim)
        self.positional_encoding = PositionalEncoding(model_dim, max_seq_length)

        # Encoder와 Decoder 레이어를 정의
        self.encoder_layers = nn.ModuleList([EncoderLayer(model_dim, num_heads, feedforward_dim, dropout) for _ in range(num_layers)])
        self.decoder_layers = nn.ModuleList([DecoderLayer(model_dim, num_heads, feedforward_dim, dropout) for _ in range(num_layers)])

        # 최종 출력을 위한 선형 변환 레이어와 Dropout을 정의
        self.fc = nn.Linear(model_dim, tgt_vocab_size)
        self.dropout = nn.Dropout(dropout)

    # 마스크를 생성하는 함수 (Decoder의 self-attention)
    def generate_mask(self, src, tgt):
        # 입력된 소스와 타겟에서 각각 0이 아닌 위치를 찾아 마스크를 생성
        # attention 스코어와 연산을 할 수 있게 하기 위해, unsqueeze를 사용하여 차원을 추가
        src_mask = (src != 0).unsqueeze(1).unsqueeze(2).to(device)
        tgt_mask = (tgt != 0).unsqueeze(1).unsqueeze(3).to(device)

        # 타겟의 시퀀스 길이를 가져옴
        seq_length = tgt.size(1)

        # nopeak_mask는 디코더가 자신보다 미래의 단어를 참조하지 못하게 하는 마스크
        # 대각선 아래쪽은 1, 위쪽은 0인 상삼각행렬을 생성하고, 이를 불리언 타입으로 변환
        nopeak_mask = (1 - torch.triu(torch.ones(1, seq_length, seq_length), diagonal=1)).bool().to(device)

        # 타겟 마스크와 nopeak_mask를 AND 연산하여 최종 타겟 마스크를 생성
        # 이 마스크는 디코더가 패딩 위치 뿐 아니라 자신보다 미래의 단어를 참조하지 못하게 함
        tgt_mask = tgt_mask & nopeak_mask

        # 소스 마스크와 타겟 마스크를 반환
        return src_mask, tgt_mask


    # 순전파 정의
    def forward(self, src, tgt):
        # 마스크 생성
        src_mask, tgt_mask = self.generate_mask(src, tgt)
        # 소스와 타겟에 각각 Embedding과 Positional Encoding을 적용
        src_embedded = self.dropout(self.positional_encoding(self.encoder_embedding(src)))
        tgt_embedded = self.dropout(self.positional_encoding(self.decoder_embedding(tgt)))

        # Encoder를 통과 (Encoder 순전파)
        encoder_output = src_embedded
        for encoder_layer in self.encoder_layers:
            encoder_output = encoder_layer(encoder_output, src_mask)

        # Decoder를 통과 (Decoder 순전파)
        decoder_output = tgt_embedded
        for decoder_layer in self.decoder_layers:
            decoder_output = decoder_layer(decoder_output, encoder_output, src_mask, tgt_mask)

        # 최종 출력 계산
        output = self.fc(decoder_output)
        return output

## 2. Transformer 모델 학습

```
💡 목차 개요 : 구현한 Transformer를 학습해봅니다
```

##### 모델 학습

데이터를 이용하여 Transformer 모델을 학습합니다

- `criterion`: 손실 함수를 정의합니다. 여기서는 CrossEntropyLoss를 사용합니다
- `optimizer`: 최적화 알고리즘을 정의합니다. 여기서는 Adam을 사용합니다
- `transformer.train()`: 모델을 학습 모드로 설정합니다

학습 과정에서 다음과 같은 단계를 거칩니다:

1. `optimizer.zero_grad()`: 그래디언트를 0으로 초기화합니다
2. `output = transformer(src_data, tgt_data[:, :-1])`: 모델에 입력 데이터를 전달하여 출력을 얻습니다
3. `loss = criterion(...)`: 손실 함수를 사용하여 손실을 계산합니다
4. `loss.backward()`: 손실에 대한 그래디언트를 계산합니다
5. `optimizer.step()`: 최적화 알고리즘을 사용하여 모델의 가중치를 업데이트합니다

In [ ]:
# 하이퍼파라미터 설정
src_vocab_size = 5000
tgt_vocab_size = 5000
model_dim = 512
num_heads = 8
num_layers = 6
feedforward_dim = 2048
max_seq_length = 100
dropout = 0.1
num_steps = 5000

# Transformer 모델을 생성
transformer = Transformer(src_vocab_size, tgt_vocab_size, model_dim, num_heads, num_layers, feedforward_dim, max_seq_length, dropout).to(device)

# 학습용 랜덤 데이터 생성
src_data = torch.randint(1, src_vocab_size, (64, max_seq_length)).to(device)  # (배치 크기, 시퀀스 길이)
tgt_data = torch.randint(1, tgt_vocab_size, (64, max_seq_length)).to(device)  # (배치 크기, 시퀀스 길이)

# 손실 함수와 최적화 알고리즘을 정의
criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

# 모델을 학습 모드로 설정
transformer.train()

# 100회의 에폭 동안 학습.
for epoch in range(100):
    # 기울기 초기화
    optimizer.zero_grad()
    # 모델의 순전파 실행
    output = transformer(src_data, tgt_data[:, :-1])
    # 손실 계산
    loss = criterion(output.contiguous().view(-1, tgt_vocab_size), tgt_data[:, 1:].contiguous().view(-1))
    # 역전파 실행
    loss.backward()
    # 가중치 업데이트
    optimizer.step()
    # 현재 에폭 및 손실 출력
    print(f"Epoch: {epoch+1}, Loss: {loss.item()}")

Epoch: 1, Loss: 8.687838554382324
Epoch: 2, Loss: 8.554313659667969
Epoch: 3, Loss: 8.485697746276855
Epoch: 4, Loss: 8.424737930297852
Epoch: 5, Loss: 8.370811462402344
Epoch: 6, Loss: 8.299156188964844
Epoch: 7, Loss: 8.222092628479004
Epoch: 8, Loss: 8.133495330810547
Epoch: 9, Loss: 8.059733390808105
Epoch: 10, Loss: 7.968536853790283
Epoch: 11, Loss: 7.884864807128906
Epoch: 12, Loss: 7.809225082397461
Epoch: 13, Loss: 7.723039627075195
Epoch: 14, Loss: 7.6345391273498535
Epoch: 15, Loss: 7.558341979980469
Epoch: 16, Loss: 7.475019454956055
Epoch: 17, Loss: 7.388033390045166
Epoch: 18, Loss: 7.313926696777344
Epoch: 19, Loss: 7.230074405670166
Epoch: 20, Loss: 7.148713111877441
Epoch: 21, Loss: 7.0703043937683105
Epoch: 22, Loss: 6.98886775970459
Epoch: 23, Loss: 6.913046360015869
Epoch: 24, Loss: 6.837372779846191
Epoch: 25, Loss: 6.772220611572266
Epoch: 26, Loss: 6.699915409088135
Epoch: 27, Loss: 6.620956897735596
Epoch: 28, Loss: 6.5430169105529785
Epoch: 29, Loss: 6.47429800